<a href="https://colab.research.google.com/github/tinetor/MLOPS_FRAUD/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dagshub mlflow

In [ ]:
import dagshub
dagshub.init(repo_owner='tiago_netor', repo_name='antifraud_ml', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=0d1bc3ed-8d49-4114-adfc-53252864d91c&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=6e8840ef714af04d7ccc4de7370e7bd4abd6330a04223ce8df29f9d02a2a56b8




Output()

Accessing as tiago_netor

Initialized MLflow to track repo "tiago_netor/antifraud_ml"

Repository tiago_netor/antifraud_ml initialized!

In [ ]:
import mlflow
import subprocess

# Specify the model URI
model_uri = "models:/MyXGBoostModel@staging"

# Get the path to the requirements file
requirements_path = mlflow.pyfunc.get_model_dependencies(model_uri, format="pip")

# Install the dependencies
subprocess.run(["pip", "install", "-r", requirements_path])


2025/02/04 11:10:10 INFO mlflow.pyfunc: To install the dependencies that were used to train the model, run the following command: '%pip install -r /tmp/tmpnaig1bmk/requirements.txt'.


CompletedProcess(args=['pip', 'install', '-r', '/tmp/tmpnaig1bmk/requirements.txt'], returncode=0)

In [ ]:
model_uri = "models:/MyXGBoostModel@staging"
model = mlflow.pyfunc.load_model(model_uri)

# Now you can evaluate the model on real data
predictions = model.predict(X_test)


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from scipy.stats import ks_2samp

def evaluate_predictions(y_true, y_pred, y_probs=None):
    """
    Evaluates binary classification predictions and prints reports.

    Args:
        y_true: True labels (1D array-like).
        y_pred: Predicted labels (1D array-like).
        y_probs: Predicted probabilities for the positive class (1D array-like, optional, required for AUC and KS).

    Returns:
        A dictionary of the calculated metrics.
    """

    cm = confusion_matrix(y_true, y_pred)
    print("Confusion Matrix:")
    print(cm)

    cr = classification_report(y_true, y_pred)
    print("\nClassification Report:")
    print(cr)

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    metrics = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "confusion_matrix": cm,
        "classification_report": cr
    }

    if y_probs is not None:
        try:
            auc = roc_auc_score(y_true, y_probs)
            metrics["auc"] = auc
            print(f"AUC: {auc}")
        except ValueError:
            print("AUC calculation failed. Ensure y_true contains at least one sample of each class.")

        try:
            ks_statistic, p_value = ks_2samp(y_probs[y_true == 1], y_probs[y_true == 0])
            metrics["ks_statistic"] = ks_statistic
            print(f"KS Statistic: {ks_statistic}")
        except ValueError:
            print("KS statistic calculation failed. Ensure y_true contains at least one sample of each class.")

    return metrics


In [ ]:
#TEST DATASET

import kagglehub

# Download latest version
path = kagglehub.dataset_download("whenamancodes/fraud-detection")

print("Path to dataset files:", path)


import pandas as pd
import os

# Assuming 'path' variable holds the correct directory path
try:
    # List files in the directory to verify
    print("Files in directory:", os.listdir(path))

    # Find the CSV file within the directory
    for filename in os.listdir(path):
        if filename.endswith(".csv"):
            csv_file_path = os.path.join(path, filename)
            break
    else:
        raise FileNotFoundError("No CSV file found in the specified directory.")

    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(csv_file_path)
    print("CSV file read successfully!")
    print(df.head())

except FileNotFoundError as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

100%|██████████| 66.0M/66.0M [00:00<00:00, 133MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/whenamancodes/fraud-detection/versions/1
Files in directory: ['creditcard.csv']
CSV file read successfully!
   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
# Define X (features) and y (target)
X = df.drop(columns=["Class"])  # Remove target column
y = df["Class"]  # Define target

# Split dataset into train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
predictions

array([1, 0, 0, ..., 0, 0, 0])

In [ ]:
 evaluate_predictions(y_test, predictions, y_probs=None)

Confusion Matrix:
[[56861     3]
 [   22    76]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.96      0.78      0.86        98

    accuracy                           1.00     56962
   macro avg       0.98      0.89      0.93     56962
weighted avg       1.00      1.00      1.00     56962



{'accuracy': 0.9995611109160493,
 'precision': 0.9620253164556962,
 'recall': 0.7755102040816326,
 'f1_score': 0.8587570621468926,
 'confusion_matrix': array([[56861,     3],
        [   22,    76]]),
 'classification_report': '              precision    recall  f1-score   support\n\n           0       1.00      1.00      1.00     56864\n           1       0.96      0.78      0.86        98\n\n    accuracy                           1.00     56962\n   macro avg       0.98      0.89      0.93     56962\nweighted avg       1.00      1.00      1.00     56962\n'}

In [ ]:
import pandas as pd

sample_input = pd.DataFrame([X_test.iloc[0]])  # Uma única amostra
try:
    pred = model.predict(sample_input)
    print(f"✅ Smoke test passou! Previsão: {pred}")
except Exception as e:
    print(f"❌ Smoke test falhou: {e}")


✅ Smoke test passou! Previsão: [1]
